In [6]:
import os
import io
import pandas as pd
from PIL import Image as Image_PIL
import shutil
from datasets import load_dataset, Dataset, Image
from tqdm import tqdm
from huggingface_hub import login
from dotenv import load_dotenv

import os

load_dotenv()  # Load environment variables from .env file


# --- Configuration ---

hf_token = os.getenv("HF_TOKEN")

hf_user = "Rajarshi-Roy-research"
# dataset_name = "Flickr30k_Grounding_Som"
# commit_message = "Updating dataset with image paths"
# img_cols = ["image","wbox_image"]
# image_to_process = "image"

dataset_name = "Som_bench_refcocog_refseg"
commit_message = "Updating dataset with VLM"
img_cols = ["image"]
image_to_process = "image"

# --- Helper Functions ---
def save_image_from_bytes(image_data, idx, image_folder):
    """Saves an image from byte data and returns the saved file path."""
    try:
        image_bytes = image_data.get("bytes", None)
        if image_bytes:
            image = Image_PIL.open(io.BytesIO(image_bytes))
            image_path = os.path.join(image_folder, f"image_{idx}.jpg")
            image.save(image_path, format="JPEG")
            print(f"✅ Saved: {image_path}")
            return image_path
        else:
            print(f"⚠️ Skipped row {idx}: No bytes found")
            return None
    except Exception as e:
        print(f"❌ Error saving image at row {idx}: {e}")
        return None


def push_df_to_hugging_face(df, img_cols, hf_user, dataset_name, commit_message, hf_token, private=False):
    """Pushes the DataFrame to the Hugging Face Hub."""
    hf_dataset = Dataset.from_pandas(df)
    
    for img_col in img_cols:
        hf_dataset = hf_dataset.cast_column(img_col, Image())

    try:
        hf_dataset.push_to_hub(
            repo_id=f"{hf_user}/{dataset_name}",
            commit_message=commit_message,
            token=hf_token,
            private=private,
        )
        print(f"✅ Dataset successfully uploaded: {hf_user}/{dataset_name}")
    except Exception as e:
        print(f"❌ Error pushing to Hugging Face Hub: {e}")




# --- Main Script ---
# Clean up existing image folder
for img_col in img_cols:
    if os.path.exists(img_col):
        shutil.rmtree(img_col)
    os.makedirs(img_col)

# Load dataset
ds = load_dataset(f"{hf_user}/{dataset_name}")
df = ds["train"].to_pandas()

for img_col in img_cols:
    # Save images and update DataFrame
    for i, row in df.iterrows():
        if isinstance(row[img_col], dict):
            df.at[i, img_col] = save_image_from_bytes(row[img_col], i, img_col)

df.to_csv("updated_dataset.csv", index=False)
print("✅ Updated DataFrame saved as 'updated_dataset.csv'!")

#Process and add captions
try:
    df = pd.read_csv("updated_dataset.csv")
except FileNotFoundError:
    pass


In [36]:

# --- Main Script ---
# Clean up existing image folder
for img_col in img_cols:
    if os.path.exists(img_col):
        shutil.rmtree(img_col)
    os.makedirs(img_col)

# Load dataset
ds = load_dataset(f"{hf_user}/{dataset_name}")
df = ds["train"].to_pandas()

for img_col in img_cols:
    # Save images and update DataFrame
    for i, row in df.iterrows():
        if isinstance(row[img_col], dict):
            df.at[i, img_col] = save_image_from_bytes(row[img_col], i, img_col)

df.to_csv("updated_dataset.csv", index=False)
print("✅ Updated DataFrame saved as 'updated_dataset.csv'!")

#Process and add captions
try:
    df = pd.read_csv("updated_dataset.csv")
except FileNotFoundError:
    pass


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

❌ Error saving image at row ['000000007601']: 'list' object has no attribute 'get'


TypeError: Provided `function` which is applied to all elements of table returns a `dict` of types [<class 'list'>, <class 'NoneType'>]. When using `batched=True`, make sure provided `function` returns a `dict` of types like `(<class 'list'>, <class 'numpy.ndarray'>, <class 'pandas.core.series.Series'>, <class 'tensorflow.python.framework.tensor.Tensor'>, <class 'torch.Tensor'>)`.

In [4]:
import torch
from transformers import Pix2StructProcessor, Pix2StructForConditionalGeneration
import requests
from PIL import Image as Image_PIL

# Set device to CUDA if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the processor and model, then move the model to the chosen device
processor = Pix2StructProcessor.from_pretrained('google/deplot')
model = Pix2StructForConditionalGeneration.from_pretrained('google/deplot').to(device)



2025-03-10 12:19:58.474292: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-10 12:19:58.484240: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741609198.498548    1371 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741609198.502794    1371 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 12:19:58.517699: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

preprocessor_config.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.62k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/851k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.88k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

In [8]:
df

,id,image,obj_text,ref_ids,json_data,depth_caption
0,7601,image/image_0.jpg,['A black cow that is only half visible standi...,[3 1 2],"[{'ref_id': 3, 'text': array(['A black cow tha...",Closest: a cow\n----\nMid Range: a power line\...
1,31187,image/image_1.jpg,['little girl in light green shirt; Little Gir...,[2 1 3 4],"[{'ref_id': 2, 'text': array(['little girl in ...",Closest: a birthday cake\n----\nMid Range: a b...
2,66669,image/image_2.jpg,['an orange cat stuffed animal; The teddy bear...,[4 5],"[{'ref_id': 4, 'text': array(['an orange cat s...",Closest: a baby\n----\nMid Range: a cow\n----\...
3,17997,image/image_3.jpg,['The giraffe standing in the middle.; A giraf...,[3 2],"[{'ref_id': 3, 'text': array(['The giraffe sta...",Closest: three giraffes\n----\nMid Range: a pe...
4,32707,image/image_4.jpg,['skier in orange coat in the right hand pictu...,[2 1],"[{'ref_id': 2, 'text': array(['skier in orange...",Closest: two people\n----\nMid Range: the Eart...
...,...,...,...,...,...,...
95,37862,image/image_95.jpg,['the little dog with black fur; smallest; bot...,[2],"[{'ref_id': 2, 'text': array(['the little dog ...",Closest: three cats\n----\nMid Range: a guitar...
96,11774,image/image_96.jpg,['man holding two plates; Short guy with smore...,[2 1],"[{'ref_id': 2, 'text': array(['man holding two...",Closest: two men\n----\nMid Range: a couple\n-...
97,7852,image/image_97.jpg,['The old school fire truck with a blue engine...,[1 2],"[{'ref_id': 1, 'text': array(['The old school ...",Closest: a red tractor\n----\nMid Range: a bla...
98,2411,image/image_98.jpg,['The white piece of cake; The slice of cake o...,[5 4],"[{'ref_id': 5, 'text': array(['The white piece...",Closest: a broken plate\n----\nMid Range: a pa...


In [9]:
import re
import ast

def extract_list_from_string(text):
    """
    Extracts a list from a string by finding the first occurrence of square brackets.

    Parameters:
        text (str): The input string containing a Python list.

    Returns:
        list: Extracted list if found, otherwise an empty list.
    """
    match = re.search(r"\[.*?\]", text, re.DOTALL)
    if match:
        try:
            return ast.literal_eval(match.group(0))  # Safely parse the list
        except (SyntaxError, ValueError):
            return []
    return []

# Example usage
text = "```python\n[3, 1, 2]\n```"
print(extract_list_from_string(text))  # Output: [3, 1, 2]
print(type(extract_list_from_string(text)))  # Output: [3, 1, 2]


[3, 1, 2]
<class 'list'>


In [ ]:

url = "https://t4.ftcdn.net/jpg/01/62/69/25/360_F_162692511_SidIKVCDnt5UKHPNqpCb2MSKvfBlx1lG.jpg"
image = Image.open(requests.get(url, stream=True).raw)
image

# Prepare inputs and move them to the device
inputs = processor(images=image, text="Generate underlying data table of the figure below:", return_tensors="pt")
inputs = {key: tensor.to(device) for key, tensor in inputs.items()}

# Generate predictions on the GPU
predictions = model.generate(**inputs, max_new_tokens=512)

# Decode and print the output
print(processor.decode(predictions[0], skip_special_tokens=True))


In [32]:
import httpx
import base64
import ast

def text_to_list(text):
    return ast.literal_eval(text)





def get_prompt(obj_list):
  prompt=f"""I have labeled a bright numeric ID at the center for each visual object in the image. \nPlease tell me the IDs for:"""

  for i in obj_list:
    prompt += f"\n-{i}\n"

  prompt+= "\nPlease, properly give answer in the form of just a python list format accoriding to the order of objects asked\nAnswer Example:`[1,2,4..]`"

  # print(prompt)
  return prompt

def get_response(prompt,img_path):
  
  image = Image_PIL.open(img_path)
  print(prompt)
  
  # Prepare inputs and move them to the device
  inputs = processor(images=image, text=prompt, return_tensors="pt")
  inputs = {key: tensor.to(device) for key, tensor in inputs.items()}

  # Generate predictions on the GPU
  predictions = model.generate(**inputs, max_new_tokens=512)

  # Decode and print the output
  response = processor.decode(predictions[0], skip_special_tokens=True)
  
  print("response",response)
  print(extract_list_from_string(response))


def get_prediction(index):

  img_path = df.loc[index,"image"]
  obj_list = text_to_list(df.loc[index,"obj_text"])

  prompt = get_prompt(obj_list)
  print(prompt)
  return obj_list

  # return get_response(prompt,img_path)


In [33]:
list_ =  get_prediction(1)
list_

I have labeled a bright numeric ID at the center for each visual object in the image. 
Please tell me the IDs for:
-little girl in light green shirt; Little Girl in a green dress waiting for the cake to be cut.; green; green dress; green shirt; Girl in green; girl in green; girl in greenthe blonde girls head; blonde hair with small green hairtie; blonde hair; blonde hair; blonde girl; blonde hair girl on left; hair in left corner; blonde headA knife cutting a cake.; Person wearing jacket cutting the cake.; standing man; person cutting cake; cutting the cake; man cutting cakegirl with glasses; glasses; girl with glasses; right kid; right gal!; right kid

Please, properly give answer in the form of just a python list format accoriding to the order of objects asked
Answer Example:`[1,2,4..]`


['little girl in light green shirt; Little Girl in a green dress waiting for the cake to be cut.; green; green dress; green shirt; Girl in green; girl in green; girl in greenthe blonde girls head; blonde hair with small green hairtie; blonde hair; blonde hair; blonde girl; blonde hair girl on left; hair in left corner; blonde headA knife cutting a cake.; Person wearing jacket cutting the cake.; standing man; person cutting cake; cutting the cake; man cutting cakegirl with glasses; glasses; girl with glasses; right kid; right gal!; right kid']

In [34]:
len(list_)

1